In [1]:
import random
from datetime import datetime, timedelta

# Define the number of rows
num_rows = 1000

# Generate dates within the last 365 days
start_date = datetime.now() - timedelta(days=365)
date_data = [(start_date + timedelta(days=random.randint(0, 365))).strftime('%Y-%m-%d') for _ in range(num_rows)]

# Generate categorical data for 'Category' and 'Status' columns
categories = ["A", "B", "C"]
status_types = ["Active", "Inactive", "Pending"]
category_data = [random.choice(categories) for _ in range(num_rows)]
status_data = [random.choice(status_types) for _ in range(num_rows)]

# Generate numerical data for 'Sales', 'Profit', and 'Quantity' columns
sales_data = [round(random.uniform(1000, 5000), 2) for _ in range(num_rows)]
profit_data = [round(random.uniform(100, 500), 2) for _ in range(num_rows)]
quantity_data = [random.randint(1, 50) for _ in range(num_rows)]

# Combine data into a list of dictionaries for easy transformation
dummy_data = [
    {
        "Date": date_data[i],
        "Category": category_data[i],
        "Status": status_data[i],
        "Sales": sales_data[i],
        "Profit": profit_data[i],
        "Quantity": quantity_data[i],
        "Discount": round(random.uniform(0, 0.5), 2)  # Generate discount as a float between 0 and 0.5
    }
    for i in range(num_rows)
]

In [2]:
import polars as pl

In [6]:
data = pl.LazyFrame(dummy_data).collect(streaming=True)
data.write_parquet('./data/sales.parquet',compression="zstd",partition_by=['Category','Status'])

In [14]:
pl.scan_parquet('./data/sales.parquet').filter((pl.col('Category') == 'B') & (pl.col('Status')=='Active')).collect()

Date,Category,Status,Sales,Profit,Quantity,Discount
str,str,str,f64,f64,i64,f64
"""2024-01-29""","""B""","""Active""",2980.41,295.57,14,0.47
"""2024-11-05""","""B""","""Active""",3522.16,223.25,35,0.05
"""2024-10-13""","""B""","""Active""",2693.31,328.58,34,0.3
"""2024-03-11""","""B""","""Active""",4167.64,493.6,44,0.39
"""2024-04-18""","""B""","""Active""",2507.12,474.24,6,0.42
…,…,…,…,…,…,…
"""2024-03-31""","""B""","""Active""",2647.91,266.16,12,0.05
"""2023-12-17""","""B""","""Active""",4460.02,476.4,34,0.06
"""2023-12-04""","""B""","""Active""",1404.45,474.81,31,0.03


In [25]:
sqll = pl.scan_parquet('./data/sales.parquet')
sql_context = pl.SQLContext()

# Register the dataframe as a table 'df'
sql_context.register("df", sqll)

# Execute the query with the registered table 'df'
filtered_data = sql_context.execute('''
    SELECT * FROM df 
    WHERE "Category" = 'B' AND "Status" = 'Active'
''')

# Print the filtered data
print(filtered_data.collect(streaming=True))

shape: (98, 7)
┌────────────┬──────────┬──────────┬──────────┬────────┬─────────┬────────┐
│ Date       ┆ Quantity ┆ Discount ┆ Category ┆ Status ┆ Sales   ┆ Profit │
│ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---    ┆ ---     ┆ ---    │
│ str        ┆ i64      ┆ f64      ┆ str      ┆ str    ┆ f64     ┆ f64    │
╞════════════╪══════════╪══════════╪══════════╪════════╪═════════╪════════╡
│ 2024-01-29 ┆ 14       ┆ 0.47     ┆ B        ┆ Active ┆ 2980.41 ┆ 295.57 │
│ 2024-11-05 ┆ 35       ┆ 0.05     ┆ B        ┆ Active ┆ 3522.16 ┆ 223.25 │
│ 2024-10-13 ┆ 34       ┆ 0.3      ┆ B        ┆ Active ┆ 2693.31 ┆ 328.58 │
│ 2024-03-11 ┆ 44       ┆ 0.39     ┆ B        ┆ Active ┆ 4167.64 ┆ 493.6  │
│ 2024-04-18 ┆ 6        ┆ 0.42     ┆ B        ┆ Active ┆ 2507.12 ┆ 474.24 │
│ …          ┆ …        ┆ …        ┆ …        ┆ …      ┆ …       ┆ …      │
│ 2024-03-31 ┆ 12       ┆ 0.05     ┆ B        ┆ Active ┆ 2647.91 ┆ 266.16 │
│ 2023-12-17 ┆ 34       ┆ 0.06     ┆ B        ┆ Active ┆ 4460.02 ┆ 476.4 